<a href="https://colab.research.google.com/github/Yopjdj/DRL-based/blob/main/minorproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

# Number of devices and tasks
num_devices = 200
num_tasks = 100

# Generate random locations for devices and tasks
def generate_random_locations(num_devices, num_tasks, filename):
    device_locations = [(random.randint(0, 500), random.randint(0, 500)) for _ in range(num_devices)]
    task_locations = [(random.randint(0, 500), random.randint(0, 500)) for _ in range(num_tasks)]

    with open(filename, 'w') as file:
        for loc in task_locations:
            file.write(f"task {loc[0]} {loc[1]}\n")
        for loc in device_locations:
            file.write(f"device {loc[0]} {loc[1]}\n")

generate_random_locations(num_devices, num_tasks, 'locations.txt')
print(f"Random locations for {num_tasks} tasks and {num_devices} devices have been saved to locations.txt")

In [ ]:
import numpy as np
import random

# Define the Q-table and parameters
num_devices = 200  # Number of devices
num_tasks = 100  # Number of tasks
Q_table = np.zeros((num_tasks, num_devices))  # Initialize the Q-table
learning_rate = 0.1
discount_factor = 0.9
exploration_rate = 0.1

#Function to read locations from a file
def read_locations(filename):
    task_locations = []
    device_locations = []
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith("task"):
                _, x, y = line.strip().split()
                task_locations.append((int(x), int(y)))
            elif line.startswith("device"):
                _, x, y = line.strip().split()
                device_locations.append((int(x), int(y)))
    return task_locations, device_locations

# Read locations from file
task_locations, device_locations = read_locations('locations.txt')

# Function to calculate the Euclidean distance between two points
def calculate_distance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

# Function to choose the next task and device based on the Q-table
def choose_action(task_index):
    if np.random.uniform() < exploration_rate:
        # Explore by choosing a random device
        device_index = np.random.randint(num_devices)
    else:
        # Exploit by choosing the device with the highest Q-value
        device_index = np.argmax(Q_table[task_index])
    return device_index

# Main training loop
for episode in range(1000):
    # Shuffle the tasks
    task_indices = list(range(num_tasks))
    random.shuffle(task_indices)

    total_distance = 0
    current_device_location = device_locations[0]  # Start from the first device location

    for task_index in task_indices:
        # Choose the device for the current task
        device_index = choose_action(task_index)

        # Calculate the distance between the current device location and the task location
        task_location = task_locations[task_index]
        distance = calculate_distance(current_device_location, task_location)
        total_distance += distance

        # Update the Q-table
        old_value = Q_table[task_index, device_index]
        remaining_tasks = task_indices[(task_indices.index(task_index) + 1):]
        if remaining_tasks:
            next_max = np.max(Q_table[remaining_tasks])
        else:
            next_max = 0  # If no remaining tasks, set next_max to 0
        new_value = (1 - learning_rate) * old_value + learning_rate * (distance + discount_factor * next_max)
        Q_table[task_index, device_index] = new_value

        # Update the current device location
        current_device_location = device_locations[device_index]

    # Print the total distance for this episode
    # print(f"Episode {episode + 1}: Total distance = {total_distance}")

# Task assignment based on the trained Q-table
task_assignments = []
for task_index in range(num_tasks):
    device_index = np.argmax(Q_table[task_index])
    task_assignments.append((task_index, device_index))

# Output the task assignments
for task_index, device_index in task_assignments:
    print(f"Task {task_index} assigned to Device {device_index}")